In [ ]:
import os
import json
import warnings
import pathlib
import copy
import shutil

import numpy as np
import matplotlib.pyplot as plt
from skimage import transform
import imageio
import nilearn.image

import nibabel

import pymedphys

In [ ]:
download_root = pymedphys._config.get_config_dir().joinpath('data', 'medical-decathlon')
download_root.mkdir(exist_ok=True, parents=True)

hash_path = download_root.joinpath('hashes.json')

In [ ]:
base_download_url = 'https://github.com/pymedphys/data/releases/download'

In [ ]:
def get_metadata(task):
    download_url = f"{base_download_url}/{task}/dataset.json"
    metadata_path = pymedphys.data_path(
        f"medical-decathlon/{task}/dataset.json",
        url=download_url,
        hash_filepath=hash_path
    )
    
    with open(metadata_path) as f:
        metadata = json.load(f)
        
    return metadata

In [ ]:
def download_task_path(task, path):
    url = f"{base_download_url}/{task}/{path[2:].replace('/', '--os.sep--')}"
    full_path = pathlib.Path(f"medical-decathlon/{task}").joinpath(path)
    
    return pymedphys.data_path(
        full_path,
        url=url,
        hash_filepath=hash_path,
        delete_when_no_hash_found=False
    )


def get_filepaths_for_task(task):
    metadata = get_metadata(task)
        
    for paths in metadata['training']:
        image_path = download_task_path(task, paths['image'])
        label_path = download_task_path(task, paths['label'])
        
        yield image_path, label_path

In [ ]:
task = 'Task06_Lung'

In [ ]:
dimension_downscale = int(8)

In [ ]:
image_path, label_path = next(gen)

In [ ]:
download_root = image_path.parent.parent.parent
new_root = download_root.joinpath("shrunk-data")
new_task_dir = new_root.joinpath(task)
new_task_dir.mkdir(exist_ok=True, parents=True)

shutil.copy(
    download_root.joinpath(task, 'dataset.json'),
    new_task_dir.joinpath('dataset.json')
)

In [ ]:
gen = get_filepaths_for_task(task)

for image_path, label_path in gen:
    image = nibabel.load(image_path)
    label = nibabel.load(label_path)
    
    target_affine = copy.copy(image.affine)
    target_affine[0,0] = target_affine[0,0] * dimension_downscale
    target_affine[1,1] = target_affine[1,1] * dimension_downscale

    print(target_affine)
    
    target_shape = (
        image.shape[0] // dimension_downscale, 
        image.shape[1] // dimension_downscale, 
        image.shape[2]
    )
    
    print(target_shape)
    
    new_image = nilearn.image.resample_img(
        image, target_affine=target_affine,
        target_shape=target_shape,
        interpolation='nearest'
    )
    
    new_label = nilearn.image.resample_img(
        label, target_affine=target_affine,
        target_shape=target_shape,
        interpolation='nearest'
    )
    
    new_image_path = new_root.joinpath(image_path.relative_to(download_root))
    new_image_path.parent.mkdir(exist_ok=True, parents=True)
    
    new_label_path = new_root.joinpath(label_path.relative_to(download_root))
    new_label_path.parent.mkdir(exist_ok=True, parents=True)
    
    nibabel.save(new_image, new_image_path)
    nibabel.save(new_label, new_label_path)